In [2]:
import boto3
import awswrangler as wr
import pandas as pd
accessKeys = pd.read_csv("../../quant-bears_accessKeys.csv")
session = boto3.Session(
	aws_access_key_id=accessKeys.loc[0, "Access key ID"],
	aws_secret_access_key=accessKeys.loc[0, "Secret access key"]
)

s3_collection_path = "s3://quant-bears-data-collection/raw-data/"

In [54]:
source = "discountingCashFlows.dcfInsiderTrading/"

df = wr.s3.read_parquet(s3_collection_path + source + "2023-09-26.parquet", boto3_session=session)

In [39]:
df["transacted_shares"] = df["Shrs._Transacted"].str.replace(",", "").astype("float")
df["owned_shares"] = df["Shrs._Owned"].str.replace(",", "").astype("float")

In [40]:
df["transacted_v_owned"] = df["transacted_shares"] / (df["owned_shares"] + 1.)

In [41]:
df["capped_transact_v_owned"] = df["transacted_v_owned"].clip(lower = 0.0, upper = 1.0)
df["capped_transact_v_owned"].describe()

count    1042.000000
mean        0.279788
std         0.396317
min         0.000000
25%         0.009188
50%         0.050609
75%         0.419872
max         1.000000
Name: capped_transact_v_owned, dtype: float64

In [42]:
df["Type"].unique()

<StringArray>
[  'P-Purchase',       'G-Gift',     'M-Exempt',       'S-Sale',
     'F-InKind',      'A-Award',      'J-Other',             '',
     'D-Return', 'C-Conversion']
Length: 10, dtype: string

In [43]:
df["Acq./Disp."].unique()

<StringArray>
['A', 'D', '']
Length: 3, dtype: string

In [44]:
df = df[df["Acq./Disp."] != ""]

In [45]:
df[df["Type"] == "S-Sale"]["ticker"].unique()

<StringArray>
[  'AEL',  'ASAN',  'CBRL',  'CELH',  'CFLT',  'CHPT',   'CNM',   'CNO',
  'COCO',  'COIN',  'COUR',  'CRDO',   'CRS',  'CRWD',   'CXM',   'DNA',
    'DV',   'EDR',   'ESI',  'FROG',  'FSLY',  'GETY',  'GSHD',  'GTLB',
  'GWRE',  'HALO',   'HCP',  'HIMS',    'HP',   'HQY',  'IBKR',   'IOT',
 'LBTYA', 'LBTYK',   'LNW',  'LPLA',  'LSCC',  'MSGE',  'MSTR',   'MUR',
   'NET',   'NUS',    'PI',  'PINS',    'PR',  'PROK',  'PTEN',  'PWSC',
  'RCUS',  'RXRX',   'SCS',  'SEIC',   'SFM',  'SGEN',   'SIG',  'SMAR',
  'SONO',  'SPLK',  'SQSP',  'STWD',  'TDOC',  'TEAM',  'TENB',  'TNET',
   'TTD',  'ZETA',   'ZIP',    'ZM',   'ZUO']
Length: 69, dtype: string

In [46]:
df[df["Type"] == "P-Purchase"]["ticker"].unique()

<StringArray>
['ABCL', 'ADC', 'ASAN', 'CHPT', 'DV', 'H', 'HPK', 'SNV', 'STAA']
Length: 9, dtype: string

In [47]:
df[df["Type"] == "P-Purchase"].sort_values("capped_transact_v_owned", ascending = False)

,ticker,Transaction_Date,Reporting_Name,Relationship,Shrs._Owned,Type,Acq./Disp.,Shrs._Transacted,Price__,Amount__,transacted_shares,owned_shares,transacted_v_owned,capped_transact_v_owned
918,STAA,2023-09-21,Lin Elizabeth Yeu,director,"2,855",P-Purchase,A,"2,500",40.00,"100,000.00",2500.0,2855.0,0.875350,0.875350
913,SNV,2023-09-19,Samaha Eli,10 percent owner,"1,598,082",P-Purchase,A,"197,705",21.40,"4,230,333.00",197705.0,1598082.0,0.123714,0.123714
1,ADC,2023-09-21,RAKOLTA JOHN JR,director,"360,056",P-Purchase,A,"30,000",57.48,"1,724,400.00",30000.0,360056.0,0.083320,0.083320
171,CHPT,2023-09-20,"Q-GRG VII (CP) Investment Partners, LLC",10 percent owner,"26,607,128",P-Purchase,A,"1,850,000",5.40,"9,990,000.00",1850000.0,26607128.0,0.069530,0.069530
476,HPK,2023-09-20,HIGHTOWER JACK,"director, 10 percent owner, officer: Chief Exe...","4,358,391",P-Purchase,A,"275,999",16.55,"4,567,728.00",275999.0,4358391.0,0.063326,0.063326
477,HPK,2023-09-19,HIGHTOWER JACK,"director, 10 percent owner, officer: Chief Exe...","4,082,392",P-Purchase,A,"224,001",16.59,"3,716,065.00",224001.0,4082392.0,0.054870,0.054870
388,H,2023-09-21,TUTTLE RICHARD C,director,"35,627",P-Purchase,A,"1,000",104.69,"104,690.00",1000.0,35627.0,0.028068,0.028068
912,SNV,2023-09-20,Samaha Eli,10 percent owner,"1,634,507",P-Purchase,A,"36,425",21.40,"779,495.00",36425.0,1634507.0,0.022285,0.022285
919,STAA,2023-09-21,"BROADWOOD PARTNERS, L.P.",10 percent owner,"9,264,427",P-Purchase,A,"154,814",39.62,"6,133,467.00",154814.0,9264427.0,0.016711,0.016711
0,ABCL,2023-09-22,Hayden Michael R,director,"1,323,397",P-Purchase,A,"20,000",4.91,"98,200.00",20000.0,1323397.0,0.015113,0.015113
